In [4]:
from __future__ import annotations

import os
import pickle
import random
import sys

import joblib
import numpy as np
import pandas as pd
import pymovements as pm
from scipy import interpolate
from tqdm import tqdm
import math

In [8]:
data_path = '../data/EHTaskDataset/RawData/User_01_Video_01_Task_1.txt'

In [22]:
def ScreenCoord2AngularCoord(ScreenCoord):
    # Inverse transformation of screen coords (0-1) to angular coords (degrees).

    # Parameters of the Hmd (HTC Vive).
    # Vertical FOV.
    VerticalFov = math.pi * 110 / 180
    # Size of a half screen.
    ScreenWidth = 1080
    ScreenHeight = 1200
    # The pixel distance between the eye and the screen center.
    ScreenDist = 0.5 * ScreenHeight / math.tan(VerticalFov / 2)

    AngularCoord = np.zeros(2)

    # The X coord.
    AngularCoord[0] = (math.atan((ScreenCoord[0] - 0.5) * ScreenWidth / ScreenDist) * 180) / math.pi
    # The Y coord.
    AngularCoord[1] = (math.atan((ScreenCoord[1] - 0.5) * ScreenHeight / ScreenDist) * 180) / math.pi

    return AngularCoord

In [23]:
def AngularCoord2ScreenCoord(AngularCoord):
	# transform the angular coords ((0 deg, 0 deg) at screen center) to screen coords which are in the range of
	# 0-1. (0, 0) at Bottom-left, (1, 1) at Top-right
	
	# the parameters of our Hmd (HTC Vive).
	# Vertical FOV.
	VerticalFov = math.pi*110/180;
	# Size of a half screen.
	ScreenWidth = 1080;
	ScreenHeight = 1200;
	# the pixel distance between eye and the screen center.
	ScreenDist = 0.5* ScreenHeight/math.tan(VerticalFov/2);
	
	ScreenCoord = np.zeros(2)
	
	# the X coord.
	ScreenCoord[0] = 0.5 + (ScreenDist * math.tan(math.pi*AngularCoord[0] / 180)) / ScreenWidth; 
	# the Y coord.
	ScreenCoord[1] = 0.5 + (ScreenDist * math.tan(math.pi*AngularCoord[1] / 180)) / ScreenHeight;
	return ScreenCoord

In [25]:
ScreenCoord = [0.58, 0.56]
AngularCoord = ScreenCoord2AngularCoord(ScreenCoord)
print(AngularCoord)
ScreenCoordNew = AngularCoord2ScreenCoord(AngularCoord)
print(ScreenCoordNew)

[11.62104404  9.72475041]
[0.58 0.56]


In [6]:
def transform_to_new_seqlen_length(X, new_seq_len, skip_padded=False):
    """
    Example: if old seq len was 7700, new_seq_len=1000:
    Input X has: 144 x 7700 x n_channels
    Output X has: 144*8 x 1000 x n_channels
    The last piece of each trial 7000-7700 gets padded with first 300 of this piece to be 1000 long
    :param X:
    :param new_seq_len:
    :return:
    """
    print(X.shape)
    print(new_seq_len)
    n, rest = np.divmod(X.shape[1], new_seq_len)

    if rest > 0 and not skip_padded:
        n_rows = X.shape[0] * (n + 1)
    else:
        n_rows = X.shape[0] * n

    X_new = np.nan * np.ones((n_rows, new_seq_len, X.shape[2]))

    idx = 0
    for t in range(0, X.shape[0]):
        for i in range(0, n):
            # cut out 1000 ms piece of trial t
            X_tmp = np.expand_dims(X[t, i * new_seq_len: (i + 1) * new_seq_len, :], axis=0)

            # concatenate pieces
            X_new[idx, :, :] = X_tmp

            idx = idx + 1

        if rest > 0 and not skip_padded:
            # concatenate last one with pad
            start_idx_last_piece = new_seq_len * (n)
            len_pad_to_add = new_seq_len - rest
            # piece to pad:
            X_incomplete = np.expand_dims(X[t, start_idx_last_piece:X.shape[1], :], axis=0)
            # padding piece:
            start_idx_last_piece = new_seq_len * (n - 1)
            X_pad = np.expand_dims(X[t, start_idx_last_piece:start_idx_last_piece + len_pad_to_add, :], axis=0)

            X_tmp = np.concatenate((X_incomplete, X_pad), axis=1)

            # concatenate last piece of original row t
            X_new[idx, :, :] = X_tmp

            idx = idx + 1

    seq_len = new_seq_len
    # print(X_new.shape)
    assert np.sum(
        np.isnan(X_new[:, :, 0])) == 0, 'Cutting into pieces failed, did not fill each position of new matrix.'

    return X_new

def deg2pix(deg, screenPX, screenCM, distanceCM):
    from math import atan2, degrees
    # Converts degrees of visual angle to pixel screen coordinate
    # screenPX is the number of pixels that the monitor has in the horizontal
    # axis (for x coord) or vertical axis (for y coord)
    # screenCM is the width of the monitor in centimeters
    # distanceCM is the distance of the monitor to the retina
    # pix: screen coordinate in pixels
    # adjust origin: if origin (0,0) of screen coordinates is in the corner of the screen rather than in the center, set to True to center coordinates
    deg = np.array(deg)

    deg_per_px = degrees(atan2(.5 * screenCM, distanceCM)) / (.5 * screenPX)
    return deg / deg_per_px


def vecvel(x, sampling_rate=1):
    # sanity check: horizontal and vertical gaze coordinates missing values at the same time (Eyelink eyetracker never records only one coordinate)
    assert np.array_equal(np.isnan(x[:, 0]), np.isnan(x[:, 1]))
    N = x.shape[0]
    v = np.zeros((N, 2))  # first column for x-velocity, second column for y-velocity
    v[1:N, ] = sampling_rate * (x[1:N, :] - x[0:N - 1, :])
    return v


In [7]:
def get_ehtask_data_for_user(data_path,
                             sampling_rate=1000,
                             transform_deg_to_px=False,
                             screen_config={'resolution': [1680, 1050],
                                            'screen_size': [47.4, 29.7],
                                            'distance': 420.12},
                             max_vel=500,
                             output_length=None,
                             delete_high_velocities=False,
                             target_sampling_rate=None):
    
    if output_length is None:
        output_length = sampling_rate
    with open(data_path, 'r') as file:
        x_vals = []
        y_vals = []
        times = []

        for line in file:
            values = line.split()

            timestamp = int(values[0])
            frame_no = int(values[1])
            x_px = float(values[4])
            y_px = float(values[5])
            [x_val, y_val] = ScreenCoord2AngularCoord([x_px, y_px])
            x_vals.append(x_val)
            y_vals.append(y_val)
            times.append(frame_no)

        if target_sampling_rate is not None:
            cur_interpolate_x = interpolate.interp1d(times, x_vals)
            cur_interpolate_y = interpolate.interp1d(times, y_vals)

            step_size = sampling_rate / target_sampling_rate
            use_time_stamps = np.arange(np.min(times), np.max(times), step_size)
            x_dva_gazebase = cur_interpolate_x(use_time_stamps)
            y_dva_gazebase = cur_interpolate_y(use_time_stamps)
            X = np.array([
                x_dva_gazebase,
                y_dva_gazebase,
            ]).T
            sampling_rate = target_sampling_rate
        else:
            X = np.array([
                x_vals,
                y_vals,
            ]).T

        # transform deg to pix
        if transform_deg_to_px:
            X_px = X.copy()
            X_px[:, 0] = deg2pix(X_px[:, 0],
                                 screen_config['resolution'][0],
                                 screen_config['screen_size'][0],
                                 screen_config['distance'])
            # adjust origin
            X_px[:, 0] += screen_config['resolution'][0] / 2

            X_px[:, 1] = deg2pix(X_px[:, 1],
                                 screen_config['resolution'][1],
                                 screen_config['screen_size'][1],
                                 screen_config['distance'])
            # adjust origin
            X_px[:, 1] += screen_config['resolution'][1] / 2
        else:
            X_px = np.zeros(X.shape)

        # transform to velocities
        vel_left = vecvel(X, sampling_rate)
        vel_left[vel_left > max_vel] = max_vel
        vel_left[vel_left < -max_vel] = -max_vel
        if delete_high_velocities:
            not_high_velocity_ids = np.logical_or(
                np.abs(vel_left[:, 0]) >= max_vel,
                np.abs(vel_left[:, 1]) >= max_vel,
            )
            X = X[not_high_velocity_ids]
            vel_left = vel_left[not_high_velocity_ids]
        X_vel = vel_left

        X_vel_transformed = transform_to_new_seqlen_length(
            X=np.reshape(X_vel, [1, X_vel.shape[0], X_vel.shape[1]]),
            new_seq_len=output_length,
            skip_padded=True,
        )

        X_deg_transformed = transform_to_new_seqlen_length(
            X=np.reshape(X, [1, X.shape[0], X.shape[1]]),
            new_seq_len=output_length,
            skip_padded=True,
        )

        X_px_transformed = transform_to_new_seqlen_length(
            X=np.reshape(X_px, [1, X_px.shape[0], X_px.shape[1]]),
            new_seq_len=output_length,
            skip_padded=True,
        )

        user_dict = {
            'X': X,
            'X_deg': X,
            'X_deg_transformed': X_deg_transformed,
            'X_px': X_px,
            'X_px_transformed': X_px_transformed,
            'X_vel': X_vel,
            'X_vel_transformed': X_vel_transformed,
            'path': data_path,

        }
        return user_dict

In [23]:
get_ehtask_data_for_user(data_path)

(1, 14997, 2)
1000
(1, 14997, 2)
1000
(1, 14997, 2)
1000


{'X': array([[1.47255904, 0.        ],
        [1.47255904, 0.        ],
        [1.47255904, 0.        ],
        ...,
        [5.87089982, 1.63609223],
        [5.87089982, 1.63609223],
        [5.87089982, 1.63609223]]),
 'X_deg': array([[1.47255904, 0.        ],
        [1.47255904, 0.        ],
        [1.47255904, 0.        ],
        ...,
        [5.87089982, 1.63609223],
        [5.87089982, 1.63609223],
        [5.87089982, 1.63609223]]),
 'X_deg_transformed': array([[[  1.47255904,   0.        ],
         [  1.47255904,   0.        ],
         [  1.47255904,   0.        ],
         ...,
         [  2.94317528,   3.26952067],
         [  2.94317528,   3.26952067],
         [  2.94317528,   3.26952067]],
 
        [[  2.94317528,   3.26952067],
         [  1.47255904,   4.89764745],
         [  1.47255904,   4.89764745],
         ...,
         [  0.        ,   3.26952067],
         [  0.        ,   3.26952067],
         [  0.        ,   3.26952067]],
 
        [[  0.        ,  

In [18]:
def load_gazebasevr_data(gaze_base_vr_dir='path_to_gazebase_data',
                       use_trial_types=['TEX', 'RAN'],  # ,'BLG','FXS','VD1','VD2','HSS']
                       target_sampling_rate=250,
                       sampling_rate=250
                       ):

    csv_files = []
    csv_files += get_csvs(gaze_base_vr_dir)

    round_list = []
    subject_list = []
    session_list = []
    trial_list = []
    path_list = []
    use_for_train = []
    for csv_file in csv_files:
        # print(csv_file)
        file_name = csv_file.split('/')[-1]
        # print(file_name)
        file_name_split = file_name.replace('.csv', '').split('_')
        cur_round = file_name_split[1][0]
        cur_subject = int(file_name_split[1][1:])
        cur_session = file_name_split[2]
        cur_trial = file_name_split[4]
        if cur_trial not in use_trial_types:
            continue
        use_for_train.append(1)
        round_list.append(cur_round)
        subject_list.append(cur_subject)
        session_list.append(cur_session)
        trial_list.append(cur_trial)
        path_list.append(csv_file)

    data_csv = pd.DataFrame({'round': round_list,
                             'subject': subject_list,
                             'session': session_list,
                             'trial': trial_list,
                             'path': path_list})

    user_data_list = []
    sub_id_list = []
    for i in tqdm(range(len(data_csv))):
        cur_line = data_csv.iloc[i]
        cur_path = cur_line['path']
        try:
            cur_sub_id = int(cur_path.split('/')[-1].split('_')[1][1:])
            cur_data = get_gazebasevr_data_for_user(cur_path,
                                         smooth=True,
                                         delete_nans=False,
                                         output_length=None,
                                         transform_deg_to_px=True,
                                         min_max_transform_px=True,
                                         target_sampling_rate=target_sampling_rate,
                                         sampling_rate=sampling_rate,
                                         )
            add_data_dict = {'x_vel_dva_left': cur_data['X_vel'][:, 0],
                             'y_vel_dva_left': cur_data['X_vel'][:, 1],
                             'x_left_px': cur_data['X_px'][:, 0],
                             'y_left_px': cur_data['X_px'][:, 1],
                             'x_dva_left': cur_data['X_deg'][:, 0],
                             'y_dva_left': cur_data['X_deg'][:, 1],
                             }

            feature_dict = {'x_vel_dva_left': 0,
                            'y_vel_dva_left': 1,
                            'x_left_px': 2,
                            'y_left_px': 3,
                            'x_dva_left': 4,
                            'y_dva_left': 5,
                            }

            cur_data_matrix = np.zeros([cur_data['X_vel'].shape[0],
                                        len(add_data_dict.keys())])

            counter = 0
            for key in add_data_dict.keys():
                cur_data_matrix[:, counter] = add_data_dict[key]
                counter += 1
            user_data_list.append(cur_data_matrix)
            sub_id_list.append(cur_sub_id)
        except Exception as e:
            print(e)
            print('error with file: ' + str(cur_path))
            break

    Y = np.zeros([len(sub_id_list), 1])
    Y[:, 0] = np.array(sub_id_list)
    y_column_dict = {'subject_id': 0,
                     }
    return user_data_list, feature_dict, Y, y_column_dict

def get_gazebasevr_data_for_user(
        data_path, max_vel=500, delete_nans=True,
        sampling_rate=250, smooth=True,
        delete_high_velocities=False,
        output_length=None,
        transform_deg_to_px=False,
        min_max_transform_px=True,
        screen_config={'resolution': [1080, 1200],
                       'screen_size': [9.2, 9.2],
                       'distance': 5},
        target_sampling_rate=None,
):
    if output_length is None:
        output_length = sampling_rate
    cur_data = pd.read_csv(data_path)
    if target_sampling_rate is not None:
        x_vals = np.array(cur_data['x'])
        y_vals = np.array(cur_data['y'])
        times = np.array(cur_data['n'])
        print(f'x_vals shape: {x_vals.shape}')
        print(f'y_vals shape: {y_vals.shape}')
        print(f'times shape: {times.shape}')
        # val = np.array(cur_data['val'])
        if delete_nans:
            nan_mask = np.isnan(x_vals) | np.isnan(y_vals)

            x_vals = x_vals[~nan_mask]
            y_vals = y_vals[~nan_mask]
            times = times[~nan_mask]
        print(f'x_vals shape: {x_vals.shape}')
        print(f'y_vals shape: {y_vals.shape}')
        print(f'times shape: {times.shape}')

        cur_interpolate_x = interpolate.interp1d(times, x_vals)
        cur_interpolate_y = interpolate.interp1d(times, y_vals)

        step_size = sampling_rate / target_sampling_rate
        use_time_stamps = np.arange(np.min(times), np.max(times), step_size)
        x_dva_gazebase = cur_interpolate_x(use_time_stamps)
        y_dva_gazebase = cur_interpolate_y(use_time_stamps)
        X = np.array([
            x_dva_gazebase,
            y_dva_gazebase,
        ]).T
        sampling_rate = target_sampling_rate
    else:
        X = np.array([
            cur_data['x'],
            cur_data['y'],
        ]).T

        # X[np.array(cur_data['val']) != 0, :] = np.nan
        # if delete_nans:
        #     not_nan_ids = np.logical_and(~np.isnan(X[:, 0]), ~np.isnan(X[:, 1]))
        #     X = X[not_nan_ids, :]

    # transform deg to pix
    if transform_deg_to_px:
        X_px = X.copy()
        X_px[:, 0] = deg2pix(X_px[:, 0],
                             screen_config['resolution'][0],
                             screen_config['screen_size'][0],
                             screen_config['distance'])
        # adjust origin
        X_px[:, 0] += screen_config['resolution'][0] / 2

        X_px[:, 1] = deg2pix(X_px[:, 1],
                             screen_config['resolution'][1],
                             screen_config['screen_size'][1],
                             screen_config['distance'])
        # adjust origin
        X_px[:, 1] += screen_config['resolution'][1] / 2
    else:
        X_px = np.zeros(X.shape)

    # transform to velocities
    vel_left = vecvel(X, sampling_rate)
    vel_left[vel_left > max_vel] = max_vel
    vel_left[vel_left < -max_vel] = -max_vel
    if delete_high_velocities:
        not_high_velocity_ids = np.logical_or(
            np.abs(vel_left[:, 0]) >= max_vel,
            np.abs(vel_left[:, 1]) >= max_vel,
        )
        X = X[not_high_velocity_ids]
        vel_left = vel_left[not_high_velocity_ids]
    X_vel = vel_left
    
    print(f'X_vel shape: {X_vel.shape}')
    print(np.where(np.isnan(X_vel)))

    X_vel_transformed = transform_to_new_seqlen_length(
        X=np.reshape(X_vel, [1, X_vel.shape[0], X_vel.shape[1]]),
        new_seq_len=output_length,
        skip_padded=True,
    )

    X_deg_transformed = transform_to_new_seqlen_length(
        X=np.reshape(X, [1, X.shape[0], X.shape[1]]),
        new_seq_len=output_length,
        skip_padded=True,
    )

    X_px_transformed = transform_to_new_seqlen_length(
        X=np.reshape(X_px, [1, X_px.shape[0], X_px.shape[1]]),
        new_seq_len=output_length,
        skip_padded=True,
    )

    user_dict = {
        'X': X,
        'X_deg': X,
        'X_deg_transformed': X_deg_transformed,
        'X_px': X_px,
        'X_px_transformed': X_px_transformed,
        'X_vel': X_vel,
        'X_vel_transformed': X_vel_transformed,
        'path': data_path,

    }
    return user_dict


In [19]:
path = '../data/GazeBaseVR/raw/data/S_1002_S1_1_VRG.csv'
get_gazebasevr_data_for_user(path,smooth=True,
                                         delete_nans=True,
                                         output_length=None,
                                         transform_deg_to_px=True,
                                         min_max_transform_px=True,
                                         target_sampling_rate=250,
                                         sampling_rate=250,
                                         )

x_vals shape: (15374,)
y_vals shape: (15374,)
times shape: (15374,)
x_vals shape: (15237,)
y_vals shape: (15237,)
times shape: (15237,)
X_vel shape: (61488, 2)
(array([], dtype=int64), array([], dtype=int64))
(1, 61488, 2)
250
(1, 61488, 2)
250
(1, 61488, 2)
250


{'X': array([[ 7.33400000e-01, -4.93970000e+00],
        [ 7.34835804e-01, -4.94400741e+00],
        [ 7.36271609e-01, -4.94831483e+00],
        ...,
        [-1.60573795e-03, -5.60645385e-01],
        [-3.02775172e-03, -5.64936374e-01],
        [-4.44976549e-03, -5.69227363e-01]]),
 'X_deg': array([[ 7.33400000e-01, -4.93970000e+00],
        [ 7.34835804e-01, -4.94400741e+00],
        [ 7.36271609e-01, -4.94831483e+00],
        ...,
        [-1.60573795e-03, -5.60645385e-01],
        [-3.02775172e-03, -5.64936374e-01],
        [-4.44976549e-03, -5.69227363e-01]]),
 'X_deg_transformed': array([[[ 0.7334    , -4.9397    ],
         [ 0.7348358 , -4.94400741],
         [ 0.73627161, -4.94831483],
         ...,
         [ 1.00172724, -4.99914511],
         [ 1.00864689, -4.99764667],
         [ 1.01151575, -4.99465198]],
 
        [[ 1.01438461, -4.99165729],
         [ 1.01725347, -4.9886626 ],
         [ 1.02056707, -4.98602023],
         ...,
         [ 0.02778038, -0.236764  ],
      